In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

In [ ]:
# Load just the blight survey data
blight_filepath = "../../data/blight_survey_data/20250527_DLBA_survey_data_UM_Detroit.xlsx"
blight_df = pd.read_excel(blight_filepath)
blight_df.columns

In [ ]:
blight_df.head()


In [ ]:
blight_df.info()



In [ ]:
blight_df.describe()



In [ ]:
blight_df.isnull().sum()

In [ ]:
blight_df['FINAL_DETERMINATION'].value_counts()

In [ ]:
blight_df['FINAL_DETERMINATION_2'].value_counts()

In [ ]:
blight_df['FIELD_DETERMINATION'].value_counts()

### Questions for blight data

1. What are the classes in blight, dependent on the above "FINAL_DETERMINATION"?
- Looks like "Other resolution pathways (salvage)" is highest blight?
- Looks like "No action" and "Vacant" are lowest blight?
- Looks like "Noticeable evidence of blight" of 26 labels, is 1?

1a. How does urbanworm do the label management here?

1b. What's the diff between FINAL_DETERMINATION, FINAL_DETERMINATION_2, and FIELD_DETERMINATION change?

2. What are the important features in predicting blight?
- Let's unpack all of the columns, seems like we could just pick 1 to start. ROOF_CONDITION?

In [ ]:
blight_df.columns

In [ ]:
blight_tight_df = blight_df[['PARCEL_ID', 'DATE_SURVEYED', 'IS_OCCUPIED', 'FIRE_DAMAGE_CONDITION', 'ROOF_CONDITION', 'OPENINGS_CONDITION', 'IS_OPEN_TO_TRESPASS', 'FIELD_DETERMINATION']]

In [ ]:
for col in blight_tight_df.columns:
    if col not in ['PARCEL_ID', 'DATE_SURVEYED']:
        print(blight_tight_df[col].value_counts())

In [ ]:
blight_labels = 